<a href="https://colab.research.google.com/github/RoshanM-18/Intermediate-Deep-Learning-projects-using-Tensorflow-Keras/blob/main/Hamlet_by_Shakespeare_Text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-1532e8ed-59a6-b0a0-3a05-4d7b63f96325)


In [ ]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import os
import requests
import sys
import random
import re
import string

### Downloading the dataset and getting one with the data

In [ ]:
response = requests.get("https://www.gutenberg.org/cache/epub/2265/pg2265.txt")

In [ ]:
data = response.text.split("\n")

In [ ]:
data = "".join("".join(data[386:]).split("\r"))

In [ ]:
data

"The Tragedie of HamletActus Primus. Scoena Prima.Enter Barnardo and Francisco two Centinels.  Barnardo. Who's there?  Fran. Nay answer me: Stand & vnfoldyour selfe   Bar. Long liue the King   Fran. Barnardo?  Bar. He   Fran. You come most carefully vpon your houre   Bar. 'Tis now strook twelue, get thee to bed Francisco   Fran. For this releefe much thankes: 'Tis bitter cold,And I am sicke at heart   Barn. Haue you had quiet Guard?  Fran. Not a Mouse stirring   Barn. Well, goodnight. If you do meet Horatio andMarcellus, the Riuals of my Watch, bid them make hast.Enter Horatio and Marcellus.  Fran. I thinke I heare them. Stand: who's there?  Hor. Friends to this ground   Mar. And Leige-men to the Dane   Fran. Giue you good night   Mar. O farwel honest Soldier, who hath relieu'd you?  Fra. Barnardo ha's my place: giue you goodnight.Exit Fran.  Mar. Holla Barnardo   Bar. Say, what is Horatio there?  Hor. A peece of him   Bar. Welcome Horatio, welcome good Marcellus   Mar. What, ha's this

In [ ]:
len(data)

157929

In [ ]:
def clean_data(text):

  tokens = text.split()
  table = str.maketrans("", "", string.punctuation)
  tokens = [x.translate(table) for x in tokens]
  tokens = [word for word in tokens if word.isalpha()]
  tokens = [x.lower() for x in tokens]
  return tokens

In [ ]:
cleaned_data = clean_data(data)

In [ ]:
cleaned_data

['the',
 'tragedie',
 'of',
 'hamletactus',
 'primus',
 'scoena',
 'primaenter',
 'barnardo',
 'and',
 'francisco',
 'two',
 'centinels',
 'barnardo',
 'whos',
 'there',
 'fran',
 'nay',
 'answer',
 'me',
 'stand',
 'vnfoldyour',
 'selfe',
 'bar',
 'long',
 'liue',
 'the',
 'king',
 'fran',
 'barnardo',
 'bar',
 'he',
 'fran',
 'you',
 'come',
 'most',
 'carefully',
 'vpon',
 'your',
 'houre',
 'bar',
 'tis',
 'now',
 'strook',
 'twelue',
 'get',
 'thee',
 'to',
 'bed',
 'francisco',
 'fran',
 'for',
 'this',
 'releefe',
 'much',
 'thankes',
 'tis',
 'bitter',
 'coldand',
 'i',
 'am',
 'sicke',
 'at',
 'heart',
 'barn',
 'haue',
 'you',
 'had',
 'quiet',
 'guard',
 'fran',
 'not',
 'a',
 'mouse',
 'stirring',
 'barn',
 'well',
 'goodnight',
 'if',
 'you',
 'do',
 'meet',
 'horatio',
 'andmarcellus',
 'the',
 'riuals',
 'of',
 'my',
 'watch',
 'bid',
 'them',
 'make',
 'hastenter',
 'horatio',
 'and',
 'marcellus',
 'fran',
 'i',
 'thinke',
 'i',
 'heare',
 'them',
 'stand',
 'whos',
 '

In [ ]:
vocab_size = len(list(set(cleaned_data)))
vocab_size

6773

In [ ]:
length = 50+1
lines = []
count = 0

for x in range(length, len(cleaned_data)):
  sequence = cleaned_data[x-length:x]
  line = " ".join(sequence)
  lines.append(line)
  count += 1

  if count>150000:
    break

In [ ]:
lines[0]

'the tragedie of hamletactus primus scoena primaenter barnardo and francisco two centinels barnardo whos there fran nay answer me stand vnfoldyour selfe bar long liue the king fran barnardo bar he fran you come most carefully vpon your houre bar tis now strook twelue get thee to bed francisco fran for'

In [ ]:
len(lines)

26646

### Converting the sequences into vector representation and creating a prefetch dataset to feed it into the RNN

In [ ]:
tokenizer = keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [ ]:
sequences[:5]

[[1,
  6768,
  4,
  6773,
  6772,
  6771,
  6769,
  455,
  2,
  1592,
  172,
  6765,
  455,
  1591,
  64,
  401,
  138,
  402,
  18,
  253,
  6760,
  80,
  457,
  201,
  252,
  1,
  27,
  401,
  455,
  457,
  24,
  401,
  6,
  49,
  59,
  6756,
  99,
  17,
  698,
  457,
  65,
  51,
  6753,
  697,
  400,
  79,
  3,
  456,
  1592,
  401,
  21],
 [6768,
  4,
  6773,
  6772,
  6771,
  6769,
  455,
  2,
  1592,
  172,
  6765,
  455,
  1591,
  64,
  401,
  138,
  402,
  18,
  253,
  6760,
  80,
  457,
  201,
  252,
  1,
  27,
  401,
  455,
  457,
  24,
  401,
  6,
  49,
  59,
  6756,
  99,
  17,
  698,
  457,
  65,
  51,
  6753,
  697,
  400,
  79,
  3,
  456,
  1592,
  401,
  21,
  16],
 [4,
  6773,
  6772,
  6771,
  6769,
  455,
  2,
  1592,
  172,
  6765,
  455,
  1591,
  64,
  401,
  138,
  402,
  18,
  253,
  6760,
  80,
  457,
  201,
  252,
  1,
  27,
  401,
  455,
  457,
  24,
  401,
  6,
  49,
  59,
  6756,
  99,
  17,
  698,
  457,
  65,
  51,
  6753,
  697,
  400,
  79,
  3,
  456,

In [ ]:
X_train, y_train = np.array(sequences)[:, :-1], np.array(sequences)[:, -1]

In [ ]:
len(X_train), len(y_train)

(26646, 26646)

In [ ]:
X_train.shape, y_train.shape

((26646, 50), (26646,))

In [ ]:
tokenizer.word_index

{'the': 1,
 'and': 2,
 'to': 3,
 'of': 4,
 'my': 5,
 'you': 6,
 'i': 7,
 'a': 8,
 'it': 9,
 'in': 10,
 'ham': 11,
 'is': 12,
 'not': 13,
 'his': 14,
 'that': 15,
 'this': 16,
 'your': 17,
 'me': 18,
 'with': 19,
 'lord': 20,
 'for': 21,
 'what': 22,
 'but': 23,
 'he': 24,
 'be': 25,
 'haue': 26,
 'king': 27,
 'so': 28,
 'as': 29,
 'him': 30,
 'will': 31,
 'our': 32,
 'we': 33,
 'no': 34,
 'on': 35,
 'are': 36,
 'shall': 37,
 'hor': 38,
 'all': 39,
 'thou': 40,
 'good': 41,
 'by': 42,
 'her': 43,
 'do': 44,
 'thy': 45,
 'from': 46,
 'then': 47,
 'if': 48,
 'come': 49,
 'or': 50,
 'now': 51,
 'hamlet': 52,
 'at': 53,
 'they': 54,
 'let': 55,
 'more': 56,
 'was': 57,
 'how': 58,
 'most': 59,
 'like': 60,
 'qu': 61,
 'know': 62,
 'laer': 63,
 'there': 64,
 'tis': 65,
 'oh': 66,
 'well': 67,
 'them': 68,
 'ophe': 69,
 'must': 70,
 'am': 71,
 'may': 72,
 'would': 73,
 'sir': 74,
 'giue': 75,
 'did': 76,
 'pol': 77,
 'vs': 78,
 'thee': 79,
 'selfe': 80,
 'hath': 81,
 'an': 82,
 'say': 83,
 'w

In [ ]:
y_train = tf.keras.utils.to_categorical(y_train, num_classes=vocab_size)

In [ ]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
X_train.shape, y_train.shape

((26646, 50), (26646, 6773))

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
dataset

<PrefetchDataset shapes: ((None, 50), (None, 6773)), types: (tf.int64, tf.float32)>

### Building a model 

In [ ]:
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, X_train.shape[1], input_length=X_train.shape[1]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(256, return_sequences=False),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dense(vocab_size, activation="softmax")
])

In [ ]:
model.compile(loss=keras.losses.CategoricalCrossentropy(), metrics=["accuracy"],
              optimizer=keras.optimizers.Adam())

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 50)            338650    
_________________________________________________________________
gru_2 (GRU)                  (None, 50, 128)           69120     
_________________________________________________________________
gru_3 (GRU)                  (None, 256)               296448    
_________________________________________________________________
dense_3 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_5 (Dense)              (None, 6773)             

In [ ]:
history = model.fit(dataset, epochs=200, steps_per_epoch=len(dataset))

Epoch 1/200
833/833 [==============================] - 12s 11ms/step - loss: 7.2067 - accuracy: 0.0302
Epoch 2/200
833/833 [==============================] - 9s 11ms/step - loss: 6.6070 - accuracy: 0.0315
Epoch 3/200
833/833 [==============================] - 9s 11ms/step - loss: 6.4111 - accuracy: 0.0317
Epoch 4/200
833/833 [==============================] - 9s 11ms/step - loss: 6.3583 - accuracy: 0.0316
Epoch 5/200
833/833 [==============================] - 9s 11ms/step - loss: 6.2824 - accuracy: 0.0307
Epoch 6/200
833/833 [==============================] - 9s 11ms/step - loss: 6.2176 - accuracy: 0.0325
Epoch 7/200
833/833 [==============================] - 9s 11ms/step - loss: 6.1295 - accuracy: 0.0341
Epoch 8/200
833/833 [==============================] - 9s 11ms/step - loss: 6.0484 - accuracy: 0.0364
Epoch 9/200
833/833 [==============================] - 9s 11ms/step - loss: 5.9842 - accuracy: 0.0415
Epoch 10/200
833/833 [==============================] - 9s 11ms/step - loss: 5.90

In [ ]:
def generate_text_sequences(text_sequence_length, seed_text, n_words):

  text = []
  seed = []

  for x in range(n_words):
    encoded = tokenizer.texts_to_sequences([seed_text])[0]
    encoded = keras.preprocessing.sequence.pad_sequences([encoded], maxlen=text_sequence_length, truncating="pre")

    pred = model.predict(encoded)
    pred = np.argmax(pred, axis=1)

    predicted_word = ""
    for word, index in tokenizer.word_index.items():
      if index==pred:
        predicted_word = word
        break

    seed_text = seed_text + " " + predicted_word
    seed.append(seed_text)
    text.append(predicted_word)

  return seed

In [ ]:
generate_text_sequences(50, lines[1], 5)

['tragedie of hamletactus primus scoena primaenter barnardo and francisco two centinels barnardo whos there fran nay answer me stand vnfoldyour selfe bar long liue the king fran barnardo bar he fran you come most carefully vpon your houre bar tis now strook twelue get thee to bed francisco fran for this vngracious',
 'tragedie of hamletactus primus scoena primaenter barnardo and francisco two centinels barnardo whos there fran nay answer me stand vnfoldyour selfe bar long liue the king fran barnardo bar he fran you come most carefully vpon your houre bar tis now strook twelue get thee to bed francisco fran for this vngracious appeare',
 'tragedie of hamletactus primus scoena primaenter barnardo and francisco two centinels barnardo whos there fran nay answer me stand vnfoldyour selfe bar long liue the king fran barnardo bar he fran you come most carefully vpon your houre bar tis now strook twelue get thee to bed francisco fran for this vngracious appeare behinde',
 'tragedie of hamletac

In [ ]:
lines[4]

'primus scoena primaenter barnardo and francisco two centinels barnardo whos there fran nay answer me stand vnfoldyour selfe bar long liue the king fran barnardo bar he fran you come most carefully vpon your houre bar tis now strook twelue get thee to bed francisco fran for this releefe much thankes'